In [1]:
!pip install datasets evaluate transformers jiwer -Uq

In [2]:
from datasets import load_dataset
dataset = load_dataset("audiofolder", data_dir="../input/voice1/dataset", split="train")

Resolving data files:   0%|          | 0/232 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files: 0it [00:00, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
import torch
from transformers import pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"

#model_name = "openai/whisper-base.en"
model_name = "openai/whisper-small.en"
#model_name = "openai/whisper-medium.en"


pipe = pipeline(
  "automatic-speech-recognition",
  model=model_name,
  chunk_length_s=30,
  device=device,
)

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [4]:
import re

def fix_reference(example):
    
    text = example['text']
    # Whisper predicts ' but on web sites can be foudn ’ (didn’t)
    text = text.replace("’","'")
    
    # Extension does not supply a dot at the end of sentence in most cases
    #if not text.endswith('.'):
    #    text += '.'
    
    # Removing brakets, otherwise text inside will be removed by normalizer
    remove_characters = '()[]{}'
    translation_table = str.maketrans('', '', remove_characters)
    text = text.translate(translation_table)
    
    # replace asdf=asdf by asdf equal asdf
    pattern = r'([a-zA-Z0-9])(=)([a-zA-Z0-9])'
    text = re.sub(pattern, r'\1 equal \3', text)
    
    # replace asdf = asdf by asdf equal asdf
    pattern = r'(.)( = )(.)'
    text = re.sub(pattern, r'\1 equal \3', text)
    
    example['text'] = text
    
    return example

In [5]:
from datasets import Audio

sampling_rate = pipe.feature_extractor.sampling_rate
dataset = dataset.cast_column("audio", Audio(sampling_rate=sampling_rate))
dataset = dataset.map(fix_reference)

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

In [6]:
def map_to_pred(batch):
    pred = pipe(batch['audio'].copy())['text']
    text = batch['text']
    return {"pred":pred,"text":text}

In [7]:
if device == "cpu":
    predictions = dataset.map(map_to_pred).remove_columns(list(set(dataset.column_names).difference({'text'})))
else:
    predicted_text = pipe(dataset['audio'].copy(), batch_size=16)
    predictions = dataset.remove_columns(list(set(dataset.column_names).difference({'text'})))
    predictions = predictions.add_column("pred",predicted_text)
    predictions = predictions.map(lambda x: {"pred":x['pred']['text']})
    predictions[0]

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

In [8]:
def map_pipe_normalizer(example):
    example['pred_default_normalize'] = pipe.tokenizer._normalize(example['pred'])
    example['text_default_normalize'] = pipe.tokenizer._normalize(example['text'])
    return example

In [9]:
predictions = predictions.map(map_pipe_normalizer)  

Map:   0%|          | 0/231 [00:00<?, ? examples/s]

In [10]:
from evaluate import load

wer = load("wer")

In [11]:
wer.compute(references=predictions["text_default_normalize"], predictions=predictions["pred_default_normalize"])

0.10558307068887889

In [12]:
wer.compute(references=predictions["text"], predictions=predictions["pred"])

0.22945446031374386

In [13]:
zero_wer_count = 0

for example in predictions:
    orig = example["text"]
    text = example["text_default_normalize"]
    pred = example["pred_default_normalize"]
    wer_score = wer.compute(references=[text], predictions=[pred])
    if wer_score == 0:
        zero_wer_count+=1
    print(wer_score)
    print(':')
    print(orig)
    print('--')
    print(text)
    print('--')
    print(pred)
    print('====')
    
print('Proportion of examples with zero wer:',zero_wer_count/len(predictions))

0.05263157894736842
:
We didn't do this here because the 🤗 Tokenizers library already uses multiple threads to tokenize our samples faster, but if you are not using a fast tokenizer backed by this library, this could speed up your preprocessing.
--
we did not do this here because the tokenizers library already uses multiple threads to tokenize our samples faster but if you are not using a fast tokenizer backed by this library this could speed up your preprocessing
--
we did not do this here because the tokenizer library already uses multiple threads to tokenize our samples faster but if you are not using a fast tokenizer backed by this library we could speed up your preprocessing
====
0.15384615384615385
:
We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding
--
we have deliberately postponed the padding to only apply it as necessary on each batch and avoid having over long inputs with a lot of pa